<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/CNN-GRU-inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, Dense, MaxPooling1D, GRU, Dropout
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [4]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv', chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [5]:
model = Sequential()
model.add(Conv1D(activation="tanh", input_shape=(len(test.columns)-1, 1), kernel_size=2, filters=16))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(activation="tanh", kernel_size=2, filters=8))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(GRU(2, activation="tanh", recurrent_activation="sigmoid"))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 69, 16)            48        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 34, 16)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 34, 16)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 33, 8)             264       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 16, 8)            0         
 1D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 16, 8)             0

In [6]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x, train_y, batch_size=32)

6161/6161 [==============================] - 27s 4ms/step - loss: 0.0968 - accuracy: 0.9765


In [7]:
test_y = test.pop('Label')
test_x = test

In [8]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [9]:
pred = model.predict(test_x, verbose=1)
pred_y = np.round(pred).astype(int)

79666/79666 [==============================] - 178s 2ms/step


In [10]:
confusion_matrix(test_y, pred_y)

array([[1224771,   30970],
       [  12936, 1280611]])

In [11]:
report = classification_report(test_y, pred_y, digits=5)
print(report)

              precision    recall  f1-score   support

           0    0.98955   0.97534   0.98239   1255741
           1    0.97639   0.99000   0.98315   1293547

    accuracy                        0.98278   2549288
   macro avg    0.98297   0.98267   0.98277   2549288
weighted avg    0.98287   0.98278   0.98277   2549288

